<h1><center>Laboratorio 8: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla y Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias y Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Benjamin Farias
- Nombre de alumno 2: Jordan Perez


### **Link de repositorio de GitHub:** `https://github.com/IsaacJordan-bit/Laboratorio-de-programaci-n-cientifica`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 28.3 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.13.1
    Uninstalling plotly-5.13.1:
      Successfully uninstalled plotly-5.13.1


https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = '/content/drive/'
except: 
    print('Ignorando conexión drive-colab')

Mounted at /content/drive


In [3]:
df_players = pd.read_csv('/content/drive/MyDrive/Proyecto/stats_players.csv')
df_players.iloc[:, 8:]

,Work_Rate,Weak_foot,Skill_Moves,Ball_Control,Dribbling,Marking,Sliding_Tackle,Standing_Tackle,Aggression,Reactions,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,High / Low,4,5,93,92,22,23,31,63,96,...,90,95,85,92,93,90,81,76,85,88
1,Medium / Medium,4,4,95,97,13,26,28,48,95,...,90,68,71,85,95,88,89,90,74,85
2,High / Medium,5,5,95,96,21,33,24,56,88,...,96,61,62,78,89,77,79,84,81,83
3,High / Medium,4,4,91,86,30,38,45,78,93,...,86,69,77,87,94,86,86,84,85,88
4,Medium / Medium,4,1,48,30,10,11,10,29,85,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Medium / Medium,1,1,12,11,12,12,13,25,48,...,27,56,14,16,13,13,11,13,15,12
17584,Medium / Medium,2,1,13,11,11,12,11,15,43,...,28,53,12,17,12,11,12,13,16,12
17585,High / Medium,2,2,44,39,40,29,35,44,42,...,54,61,41,44,28,42,35,36,42,37
17586,Medium / Medium,3,1,17,13,11,11,14,23,46,...,34,48,15,23,14,12,13,12,24,12


## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [4]:
# Generar las etiquetas para la clasificación
df_players["Label"] = df_players["National_Position"].apply(lambda x: "Seleccionado" if pd.notnull(x) else "No seleccionado")

# Contar los ejemplos por clase
class_counts = df_players["Label"].value_counts()
print(class_counts)
df_players["Label"] = df_players["Label"].apply(lambda x: 1 if x == "Seleccionado" else 0)

No seleccionado    16513
Seleccionado        1075
Name: Label, dtype: int64


In [ ]:
df_players.isna().sum()

Name                     0
Nationality              0
National_Position    16513
Club_Position            1
Height                   0
Weight                   0
Preffered_Foot           0
Age                      0
Work_Rate                0
Weak_foot                0
Skill_Moves              0
Ball_Control             0
Dribbling                0
Marking                  0
Sliding_Tackle           0
Standing_Tackle          0
Aggression               0
Reactions                0
Interceptions            0
Vision                   0
Composure                0
Crossing                 0
Short_Pass               0
Long_Pass                0
Acceleration             0
Speed                    0
Stamina                  0
Strength                 0
Balance                  0
Agility                  0
Jumping                  0
Heading                  0
Shot_Power               0
Finishing                0
Long_Shots               0
Curve                    0
Freekick_Accuracy        0
P

In [5]:
df_players = df_players[df_players.Club_Position.notna()].copy()

Se observa un evidente desequilibrio en los datos, lo cual puede ser problemático al entrenar modelos de aprendizaje supervisado.
Esto puede generar problemas durante el entrenamiento de modelos de aprendizaje supervisado. Algunos de los desafíos asociados con este desbalance son:

1. Sesgo de clase: El modelo puede tener una tendencia a favorecer la clase mayoritaria y clasificar erróneamente ejemplos de la clase minoritaria.

2. Baja precisión en la clase minoritaria: Si la clase minoritaria contiene ejemplos importantes o de interés particular, como en este caso los jugadores seleccionados, es posible que el modelo no logre identificarlos correctamente debido a la falta de ejemplos de esa clase.

3. Generalización deficiente: El modelo puede tener dificultades para generalizar y capturar patrones de ambas clases debido a la falta de representación adecuada de la clase minoritaria.

4. Evaluación engañosa: Si se evalúa el modelo utilizando métricas como la precisión, que no tienen en cuenta el desbalance de clases, puede dar una impresión engañosa del rendimiento real del modelo.

Para abordar este desbalance, existen técnicas como el muestreo estratificado, la generación de datos sintéticos, la ponderación de clases y el ajuste de umbrales de decisión. Estas estrategias pueden ayudar a mitigar el problema y mejorar el rendimiento del modelo en la clasificación de jugadores seleccionados.

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [x] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [x] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [ ] Crear 3 ``pipelines`` con diferentes clasificadores.
- [ ] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [ ] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

In [220]:
# Definir las transformaciones para cada columna

preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', MinMaxScaler(), ['Height', 'Weight', 'Age', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
                                       'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision', 'Composure',
                                       'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration', 'Speed', 'Stamina', 'Strength',
                                       'Balance', 'Agility', 'Jumping', 'Heading', 'Shot_Power', 'Finishing', 'Long_Shots',
                                       'Curve', 'Freekick_Accuracy', 'Penalties', 'Volleys']),
        ('categorical', OneHotEncoder(handle_unknown='ignore', sparse_output=False), ['Club_Position', 'Work_Rate', 'Preffered_Foot',
                                                                                      'Weak_foot', 'Skill_Moves'])
    ], sparse_threshold=0, verbose_feature_names_out=False)

preprocessor.set_output(transform='pandas')

ColumnTransformer(sparse_threshold=0,
                  transformers=[('numeric', MinMaxScaler(),
                                 ['Height', 'Weight', 'Age', 'Ball_Control',
                                  'Dribbling', 'Marking', 'Sliding_Tackle',
                                  'Standing_Tackle', 'Aggression', 'Reactions',
                                  'Interceptions', 'Vision', 'Composure',
                                  'Crossing', 'Short_Pass', 'Long_Pass',
                                  'Acceleration', 'Speed', 'Stamina',
                                  'Strength', 'Balance', 'Agility', 'Jumping',
                                  'Heading', 'Shot_Power', 'Finishing',
                                  'Long_Shots', 'Curve', 'Freekick_Accuracy',
                                  'Penalties', ...]),
                                ('categorical',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 ['Club_Position', 'Work_Rate',
                                  'Preffered_Foot', 'Weak_foot',
                                  'Skill_Moves'])],
                  verbose_feature_names_out=False)

In [56]:
# Aplicar el preprocesamiento a los datos
df_players_preprocessed = preprocessor.fit_transform(df_players)

In [57]:
df_players_preprocessed

,Height,Weight,Age,Ball_Control,Dribbling,Marking,Sliding_Tackle,Standing_Tackle,Aggression,Reactions,...,Weak_foot_1,Weak_foot_2,Weak_foot_3,Weak_foot_4,Weak_foot_5,Skill_Moves_1,Skill_Moves_2,Skill_Moves_3,Skill_Moves_4,Skill_Moves_5
0,0.576923,0.516129,0.500000,0.977778,0.946237,0.213483,0.200000,0.314607,0.648936,1.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.288462,0.387097,0.400000,1.000000,1.000000,0.112360,0.233333,0.280899,0.489362,0.985075,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.365385,0.322581,0.266667,1.000000,0.989247,0.202247,0.311111,0.235955,0.574468,0.880597,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,0.519231,0.596774,0.433333,0.955556,0.881720,0.303371,0.366667,0.471910,0.808511,0.955224,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.730769,0.709677,0.466667,0.477778,0.279570,0.078652,0.066667,0.078652,0.287234,0.835821,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,0.538462,0.548387,0.066667,0.077778,0.075269,0.101124,0.077778,0.112360,0.244681,0.283582,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
17584,0.576923,0.516129,0.066667,0.088889,0.075269,0.089888,0.077778,0.089888,0.138298,0.208955,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
17585,0.346154,0.209677,0.033333,0.433333,0.376344,0.415730,0.266667,0.359551,0.446809,0.194030,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
17586,0.480769,0.516129,0.133333,0.133333,0.096774,0.089888,0.066667,0.123596,0.223404,0.253731,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


**Respuesta:**
Para las variables numéricas, dado que son discretas y con tal de mantener su distribución relativa, se aplicón MinMaxScaler, reescalandolas al rango $[0,1]$.


Por otro lado, el uso de One-Hot Encoding en las columnas a las que se les aplicó dicho procesamiento, son categóricas, y tiene los siguientes beneficios:

1. Nos permite preservar la naturaleza categórica de estas variables al crear columnas binarias separadas para cada categoría.

2. Se evita cualquier suposición incorrecta sobre el orden o la jerarquía entre las categorías.

3. Al aplicar One-Hot Encoding, evitamos cualquier distorsión y aseguramos que cada categoría se represente de manera adecuada sin afectar su significado original.

4. El modelo de aprendizaje automático puede capturar y aprender las relaciones entre las diferentes categorías de manera más precisa. Cada categoría tiene su propia columna, lo que permite que el modelo tome decisiones basadas en la presencia o ausencia de una categoría en particular.

Se eliminan los atributos nombre y nacionalidad debido a que no son necesarios para la clasificación.

In [61]:
# pipes
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

pipe1 = Pipeline([('pross', preprocessor),
                 ('linearSVM', LinearSVC(random_state=0, tol=1e-05))])

pipe2 = Pipeline([('pross', preprocessor),
                 ('KNeighbors', KNeighborsClassifier(n_neighbors=10))])

pipe3 = Pipeline([('pross', preprocessor),
                 ('SVM', SVC(gamma='auto'))])

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [62]:
X_train, X_test, y_train, y_test = train_test_split(df_players, df_players['Label'], test_size=0.3, stratify=df_players['Label'])

In [63]:
pipe1.fit(X_train, y_train)

Pipeline(steps=[('pross',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('numeric', MinMaxScaler(),
                                                  ['Height', 'Weight', 'Age',
                                                   'Ball_Control', 'Dribbling',
                                                   'Marking', 'Sliding_Tackle',
                                                   'Standing_Tackle',
                                                   'Aggression', 'Reactions',
                                                   'Interceptions', 'Vision',
                                                   'Composure', 'Crossing',
                                                   'Short_Pass', 'Long_Pass',
                                                   'Acceleration', 'Speed',
                                                   'Stamina', 'Strength',
                                                   'Balance', 'Agility',
                                                   'Jumping', 'Heading',
                                                   'Shot_Power', 'Finishing',
                                                   'Long_Shots', 'Curve',
                                                   'Freekick_Accuracy',
                                                   'Penalties', ...]),
                                                 ('categorical',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['Club_Position', 'Work_Rate',
                                                   'Weak_foot',
                                                   'Skill_Moves'])],
                                   verbose_feature_names_out=False)),
                ('linearSVM', LinearSVC(random_state=0, tol=1e-05))])

In [64]:
pipe2.fit(X_train, y_train)

Pipeline(steps=[('pross',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('numeric', MinMaxScaler(),
                                                  ['Height', 'Weight', 'Age',
                                                   'Ball_Control', 'Dribbling',
                                                   'Marking', 'Sliding_Tackle',
                                                   'Standing_Tackle',
                                                   'Aggression', 'Reactions',
                                                   'Interceptions', 'Vision',
                                                   'Composure', 'Crossing',
                                                   'Short_Pass', 'Long_Pass',
                                                   'Acceleration', 'Speed',
                                                   'Stamina', 'Strength',
                                                   'Balance', 'Agility',
                                                   'Jumping', 'Heading',
                                                   'Shot_Power', 'Finishing',
                                                   'Long_Shots', 'Curve',
                                                   'Freekick_Accuracy',
                                                   'Penalties', ...]),
                                                 ('categorical',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['Club_Position', 'Work_Rate',
                                                   'Weak_foot',
                                                   'Skill_Moves'])],
                                   verbose_feature_names_out=False)),
                ('KNeighbors', KNeighborsClassifier(n_neighbors=10))])

In [65]:
pipe3.fit(X_train, y_train)

Pipeline(steps=[('pross',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('numeric', MinMaxScaler(),
                                                  ['Height', 'Weight', 'Age',
                                                   'Ball_Control', 'Dribbling',
                                                   'Marking', 'Sliding_Tackle',
                                                   'Standing_Tackle',
                                                   'Aggression', 'Reactions',
                                                   'Interceptions', 'Vision',
                                                   'Composure', 'Crossing',
                                                   'Short_Pass', 'Long_Pass',
                                                   'Acceleration', 'Speed',
                                                   'Stamina', 'Strength',
                                                   'Balance', 'Agility',
                                                   'Jumping', 'Heading',
                                                   'Shot_Power', 'Finishing',
                                                   'Long_Shots', 'Curve',
                                                   'Freekick_Accuracy',
                                                   'Penalties', ...]),
                                                 ('categorical',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['Club_Position', 'Work_Rate',
                                                   'Weak_foot',
                                                   'Skill_Moves'])],
                                   verbose_feature_names_out=False)),
                ('SVM', SVC(gamma='auto'))])

In [ ]:
print(classification_report(y_test, pipe1.predict(X_test)))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      4954
           1       0.67      0.01      0.01       323

    accuracy                           0.94      5277
   macro avg       0.80      0.50      0.49      5277
weighted avg       0.92      0.94      0.91      5277



In [ ]:
print(classification_report(y_test, pipe2.predict(X_test)))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      4954
           1       0.55      0.04      0.07       323

    accuracy                           0.94      5277
   macro avg       0.74      0.52      0.52      5277
weighted avg       0.92      0.94      0.91      5277



In [ ]:
print(classification_report(y_test, pipe3.predict(X_test)))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      4954
           1       0.00      0.00      0.00       323

    accuracy                           0.94      5277
   macro avg       0.47      0.50      0.48      5277
weighted avg       0.88      0.94      0.91      5277



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Respuesta**:

```
- Precision mide el ratio de clasificaciones positivas que corresponden efectivamente a la clase positiva
- Recall mide cuantos elementos de la clase positiva fueron clasificados correctamente
- Accuracy mide el total de elementos clasificados correctamente

El desbalance de clase influye inflando el accuracy al mostrar preferencia por la clase mayoritaria.
Al ver muchos más ejemplos de una clase los modelos pueden tender a optar arbitrariamente por la misma.

Los resultados obtenidos por los 3 clasificadores son bastante malos, con valores de recall cercanos a 0
para la clase positiva. De los altos niveles de accuracy se deduce que los clasificsadores tienden a
clasificar todos los casos como 0.

Los resultados se podrían mejorar aplicando alguna tecnica de balance de clases.
```

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [ ] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [ ] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

In [ ]:
df_players['Club_Position']

0         LW
1         RW
2         LW
3         ST
4         GK
        ... 
16791    Res
16902    Res
16916    Res
17113    Res
17129    Res
Name: Club_Position, Length: 1075, dtype: object

In [6]:
pl = {'ataque' : ['ST', 'CF'],
      'central_ataque' : ['RW', 'CAM', 'LW'],
      'central' : ['RM', 'CM', 'LM'],
      'central_defensa' : ['RWB', 'CDM', 'LWB'],
      'defensa' : ['RB', 'CB', 'LB'],
      'arquero' : ['GK']}

df_players['PosLabel'] = np.full(len(df_players), 'None')

for key in pl.keys():
    b = df_players['Club_Position'].isin(pl[key])
    nc = b.sum()
    df_players.loc[b, ['PosLabel']] = np.full(nc, key)
    print(key, ': ', nc)

ataque :  430
central_ataque :  581
central :  907
central_defensa :  209
defensa :  1180
arquero :  632


In [ ]:
len(df_players[df_players['Club_Position']=='Sub'])

7492

In [ ]:
len(df_players[df_players['Club_Position']=='Res'])

3146

In [66]:
preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', MinMaxScaler(), ['Height', 'Weight', 'Age', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
                                       'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision', 'Composure',
                                       'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration', 'Speed', 'Stamina', 'Strength',
                                       'Balance', 'Agility', 'Jumping', 'Heading', 'Shot_Power', 'Finishing', 'Long_Shots',
                                       'Curve', 'Freekick_Accuracy', 'Penalties', 'Volleys']),
        ('categorical', OneHotEncoder(handle_unknown='ignore', sparse_output=False), ['Work_Rate', 'Preffered_Foot', 'Weak_foot', 'Skill_Moves'])
    ], sparse_threshold=0, verbose_feature_names_out=False)

pipe4 = Pipeline([('pross', preprocessor),
                 ('KNeighbors', KNeighborsClassifier(n_neighbors=10))])

df = df_players[df_players['PosLabel'] != 'None']

X_train, X_test, y_train, y_test = train_test_split(df, df['PosLabel'], test_size=0.3, stratify=df['PosLabel'])

In [67]:
pipe4.fit(X_train, y_train)

Pipeline(steps=[('pross',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('numeric', MinMaxScaler(),
                                                  ['Height', 'Weight', 'Age',
                                                   'Ball_Control', 'Dribbling',
                                                   'Marking', 'Sliding_Tackle',
                                                   'Standing_Tackle',
                                                   'Aggression', 'Reactions',
                                                   'Interceptions', 'Vision',
                                                   'Composure', 'Crossing',
                                                   'Short_Pass', 'Long_Pass',
                                                   'Acceleration', 'Speed',
                                                   'Stamina', 'Strength',
                                                   'Balance', 'Agility',
                                                   'Jumping', 'Heading',
                                                   'Shot_Power', 'Finishing',
                                                   'Long_Shots', 'Curve',
                                                   'Freekick_Accuracy',
                                                   'Penalties', ...]),
                                                 ('categorical',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['Work_Rate', 'Weak_foot',
                                                   'Skill_Moves'])],
                                   verbose_feature_names_out=False)),
                ('KNeighbors', KNeighborsClassifier(n_neighbors=10))])

In [ ]:
print(classification_report(y_test, pipe4.predict(X_test)))

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       190
         ataque       0.77      0.76      0.76       129
        central       0.53      0.64      0.58       272
 central_ataque       0.49      0.22      0.31       174
central_defensa       0.50      0.13      0.20        63
        defensa       0.77      0.97      0.86       354

       accuracy                           0.72      1182
      macro avg       0.68      0.62      0.62      1182
   weighted avg       0.70      0.72      0.69      1182



**Respuesta**:

```
El modelo logra clasificar perfectamente a la clase arquero, con un
desempeño medio a las clases ataque y defensa, y falla al distinguir
las clases central, posiblemente por confusión entre las misas.

Se podría intentar mejorar los resultados porbando otros clasificadorer
o separando el proceso en etapas que permitan difirenciar mejor a las
sub-clases de central.
```

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [ ] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

In [7]:
dfs = pd.read_csv('/content/drive/MyDrive/Proyecto/salarios.csv')
del dfs['Unnamed: 0']

In [ ]:
dfs

,Player,Weekly Salary
0,Lionel Messi,1538905.0
1,Cristiano Ronaldo,1248536.0
2,Antoine Griezmann,997032.0
3,Neymar,797726.0
4,Gareth Bale,607706.0
...,...,...
2529,Simone Ghidotti,283.0
2530,Valentin Gendrey,100.0
2531,John Yeboah,70.0
2532,Antonis Aidonis,70.0


In [8]:
dfs = dfs.merge(df_players, left_on='Player', right_on='Name', how='inner')

In [194]:
preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', MinMaxScaler(), ['Height', 'Weight', 'Age', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
                                       'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision', 'Composure',
                                       'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration', 'Speed', 'Stamina', 'Strength',
                                       'Balance', 'Agility', 'Jumping', 'Heading', 'Shot_Power', 'Finishing', 'Long_Shots',
                                       'Curve', 'Freekick_Accuracy', 'Penalties', 'Volleys']),
        ('categorical', OneHotEncoder(handle_unknown='ignore', sparse_output=False), ['PosLabel', 'Work_Rate', 'Preffered_Foot',
                                                                                      'Weak_foot', 'Skill_Moves'])
    ], sparse_threshold=0, verbose_feature_names_out=False)

In [11]:
def r2(y_true, y_pred):
    ym = np.mean(y_true)
    e2 = ((y_true - y_pred)**2).sum()
    return 1 - e2/((y_true - ym)**2).sum()

In [221]:
from sklearn.linear_model import Lasso

X_train, X_test, y_train, y_test = train_test_split(dfs, dfs['Weekly Salary'], test_size=0.2, random_state=30)
pipe5 = Pipeline([('pross', preprocessor),
                 ('Lasso', Lasso(alpha=130))])
pipe5.fit(X_train, y_train)

Pipeline(steps=[('pross',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('numeric', MinMaxScaler(),
                                                  ['Height', 'Weight', 'Age',
                                                   'Ball_Control', 'Dribbling',
                                                   'Marking', 'Sliding_Tackle',
                                                   'Standing_Tackle',
                                                   'Aggression', 'Reactions',
                                                   'Interceptions', 'Vision',
                                                   'Composure', 'Crossing',
                                                   'Short_Pass', 'Long_Pass',
                                                   'Acceleration', 'Speed',
                                                   'Stamina', 'Strength',
                                                   'Balance', 'Agility',
                                                   'Jumping', 'Heading',
                                                   'Shot_Power', 'Finishing',
                                                   'Long_Shots', 'Curve',
                                                   'Freekick_Accuracy',
                                                   'Penalties', ...]),
                                                 ('categorical',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['Club_Position', 'Work_Rate',
                                                   'Preffered_Foot',
                                                   'Weak_foot',
                                                   'Skill_Moves'])],
                                   verbose_feature_names_out=False)),
                ('Lasso', Lasso(alpha=130))])

In [218]:
yp = pipe5.predict(X_test)

In [219]:
r2(y_test, yp)

0.353758494157698

**Respuesta**:

La métrica $R^2$ corresponde a 1 menos el ratio de la suma de error cuadrático del modelo por sobre uno que entrega siempore la media, esto se puede interpretar como que tan mejor es el modelo en comparación a dicho baseline. El mejor puntaje posible de la métrica es 1 (error igual a 0) y puede tomar valores negativos en el peor caso.

Los resultados se podrían intentar mejorar con un EDA y preprocesamiento más detallado, también sería útil disponer de una muestra de salarios más grande y tener en cuenta los outliers presentes en los datos.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>